In [ ]:
import requests
from bs4 import BeautifulSoup

# ニュース一覧ページのURL
url = "https://www.kiramex.com/news/"

# ページのHTMLを取得
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字コードを自動判別して設定

# HTMLをBeautifulSoupでパース
soup = BeautifulSoup(response.text, "html.parser")

# ニュース一覧の情報を抽出
news_list = soup.find_all("li", class_="news-block")

# ニュース情報の取得
for news in news_list:
    # ニュースの日付を取得
    date = news.find("p", class_="date").get_text(strip=True) if news.find("p", class_="date") else "N/A"
    
    # ニュースのタイトルを取得
    title = news.find("a", class_="title").get_text(strip=True)
    
    # ニュースのリンクを取得
    link = news.find("a", class_="title")["href"]

    print(f"Date: {date}")
    print(f"Title: {title}")
    print(f"Link: {link}")
    print("-" * 40)

In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
# ニュース一覧ページのURL
url = "https://www.kiramex.com/news/"

# ページのHTMLを取得
response = requests.get(url)
response.encoding = response.apparent_encoding  # 文字コードを自動判別して設定

# HTMLをBeautifulSoupでパースし、body部分を取り出す
soup = BeautifulSoup(response.text, "html.parser")
body_html = str(soup.body)  # body部分のHTMLを文字列として取得
print(body_html) # 結果を表示して確認

In [ ]:
# LLMにニュース一覧を抽出させるプロンプトを作成
prompt = f"""
以下のHTMLから最新のニュースを抽出し、「日付、タイトル、リンク」の形式で一覧を出力してください。一覧以外は出力しないでください。

# 出力様式：
Date: 日付
Title: タイトル
Link: リンク
--------------------

#HTML:
{body_html[:5000]}
"""

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": prompt},
    ],
    max_tokens=500,
    temperature=0.3
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())
